# Übung Datenbanken: Ein Zugauskunfssystem
![](https://www.klipfel-mathys.ch/images/imagesCAUM394L.jpg)

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

!wget -qOzugauskunft-kemper.db 'https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=zugauskunft-kemper.db'

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
%sql sqlite:///zugauskunft-kemper.db
%sql PRAGMA foreign_keys = ON

# Aufgabe

Nachdem wir in der letzten Aufgabe unsere Daten in folgendes Datenmodell importiert haben:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-er.png)

Wollen wir in dieser Aufgabe einige Abfragen auf diesem Modell entwerfen.

In [ ]:
!echo ".schema" | sqlite3 zugauskunft-kemper.db

### Abfrage 1: Abfahrten in Deggendorf

Entwerfen Sie eine Abfrage, die alle Abfahrten in Deggendorf anzeigt. Geben Sie die Zugnummer, den Namen, den Start- und Endbahnhof sowie die Abfahrtszeit aus.

In [ ]:
%sql SELECT ZugNr,MIN(Abfahrt) as Abfahrt FROM verbindet GROUP BY ZugNr LIMIT 10;

In [ ]:
%%sql
SELECT Zug.ZugNr, Zug.Name, verbindet.Abfahrt, start.Name as Startbahnhof, end.Name as Endbahnhof
FROM Zug
JOIN Bahnhof AS start
ON(Zug.StartBhf = start.BahnhofId)
JOIN Bahnhof AS end
ON(Zug.EndBhf = end.BahnhofId)
JOIN (SELECT ZugNr,MIN(Abfahrt) as Abfahrt FROM verbindet GROUP BY ZugNr)
AS verbindet
USING(ZugNr)
WHERE Startbahnhof LIKE 'Deggendorf%'
ORDER BY Abfahrt ASC;



Sie finden in den Ergebnissen sicher eine Zeilen mit : Abfahrt 25:27. Das ist typisch für Bahn Systeme, kann aber manche Zeitfunktion Ihrer Datenbank aus dem Tritt bringen.

## Aufgabe 2: Sortierng

## Abfrage 3: Laufweg einer zufälligen Fahrt

Geben Sie alle Halte einer zufälligen (Hint: Subselect mit RANDOM()) Fahrt aus.

### Abfrage 3: Von Deggendorf nach Grafenau

Wie komme ich von Deggendorf nach Grafenau? Gibt es einen einfachen Weg das in einer SQL Abfrage abzubilden?